# Project 3: Reddit Posts Classification

## NoteBook Contents
- Part 1 - Extraction, Cleansing
- Part 2 - EDA, Pre-Proccessing, and Modeling

----------------------------------------

### Part 1 - Extraction, Cleansing

In [12]:
#libraries

#################################
import folium
import glob
import html
import imp
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt
import numpy as np
import nltk
import os
import pandas as pd
import pandas_datareader as pdr
import pickle
import plotly
import plotly.express as px
import random as rand
import re
import requests
import scipy.stats as stats
import seaborn as sns
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
import spacy
import statsmodels.api as sm
import streamlit as st
import sympy as sy
import time

#######################
from alpha_vantage.timeseries import TimeSeries
from bs4 import BeautifulSoup
from category_encoders import OneHotEncoder as OHE
from IPython.display import display
from IPython.display import HTML
#from ipywidgets import *
from ipywidgets import interact
from math import pi
from math import sqrt
from mpl_toolkits.mplot3d import Axes3D
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from pandas import DataFrame
from pandas import Series
from pandas import json_normalize
from pathlib import Path
#from plotter import SVMPlotter
from random import sample
from scipy.stats import ttest_ind
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.weightstats import CompareMeans
from statsmodels.stats.weightstats import DescrStatsW
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#######################
from sklearn import svm
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.datasets.samples_generator import make_blobs
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.ensemble import HistGradientBoostingClassifier
#from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.inspection import plot_partial_dependence
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
#from sklearn.model_selection import GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.svm import SVR
from sklearn.svm import SVC

In [13]:
#api call function applied to both subreddits used for the project
#source: 
    # 1) https://rdrr.io/github/jfeldhege/redditoR/man/get_posts.html
    # 2) https://stackoverflow.com/questions/48572494/structuring-api-calls-in-python
    # 4) https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    # 3) seperate breakout playground session walking through how to utilize Pushshift API

def api_call(subred,iterations,current_epoch_time):
    pass
    #establish empty DataFrame
    list_df = []
    #url for Pushshift API subreddit
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit='
    #pull posts before specific date/time
    time = current_epoch_time
    
    #identify params for url
    for i in range(iterations):
        res=requests.get(
            url, 
            params={
                'subreddit': subred,
                'size': 500,
                'lang': True,
                'before': time
            }
        )
        
        data = res.json()
        posts = data['data']
        df = json_normalize(data,'data')
        
        #pull specific columns
        df = df.loc[:,['title','author','selftext','subreddit','score','created_utc','id']]
        #append DataFrame
        list_df.append(df)
        
        #add wait time
        sleeptime = np.random.randint(1, 3)
        time.sleep(sleeptime)
        
        #establish current time
        time = df['created_utc'].min()
    
    return pd.concat(list_df, axis = 0)

I switched the code cells below to raw-type cells because the api_call function takes forever to run. I exported the DataFrames as CSV files after original run

----------------------------------------

#### Cleansing

In [14]:
#pull back in csv files reflecting both api_call output DataFrames for the two subreddits

fantasy_df = pd.read_csv('../datasets/df_fantasy.csv', keep_default_na=False, index_col=0)

In [15]:
fantasy_df.head(2)

,title,author,selftext,subreddit,score,created_utc,id,is_fantasyfootball
0,NFL on Twitter- Falcons hire Arthur Smith as n...,TheFFnerd,,fantasyfootball,4,1610747812,ky4x5u,1
1,#Seahawks WR Josh Gordon's conditional reinsta...,[deleted],[deleted],fantasyfootball,1,1610746600,ky4i5o,1


In [16]:
sports_df = pd.read_csv('../datasets/df_sports.csv', keep_default_na=False, index_col=0)

In [17]:
sports_df.head(2)

,title,author,selftext,subreddit,score,created_utc,id,is_fantasyfootball
0,What does this mean?,TheRegal6,I got an email saying “We just need you to wri...,dfsports,1,1610747631,ky4uur,0
1,BEST DraftKings &amp; FanDuel NBA DFS Core 4 P...,dfsndonuts,Tonight’s Best [DraftKings and FanDuel NBA DF...,dfsports,2,1610743477,ky3erf,0


In [18]:
#merge both subreddit outputs
combined_df = fantasy_df.append(sports_df,sort=False)

In [19]:
#export merged DataFrame to csv file to save copy
combined_df.to_csv('../datasets/combined_df.csv')

In [20]:
#pull back in merged DataFrame
final_df = pd.read_csv('../datasets/combined_df.csv', keep_default_na=False, index_col=0)

In [21]:
final_df.shape

(10000, 8)

In [22]:
final_df.head(2)

,title,author,selftext,subreddit,score,created_utc,id,is_fantasyfootball
0,NFL on Twitter- Falcons hire Arthur Smith as n...,TheFFnerd,,fantasyfootball,4,1610747812,ky4x5u,1
1,#Seahawks WR Josh Gordon's conditional reinsta...,[deleted],[deleted],fantasyfootball,1,1610746600,ky4i5o,1


In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 99
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               10000 non-null  object
 1   author              10000 non-null  object
 2   selftext            10000 non-null  object
 3   subreddit           10000 non-null  object
 4   score               10000 non-null  int64 
 5   created_utc         10000 non-null  int64 
 6   id                  10000 non-null  object
 7   is_fantasyfootball  10000 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 703.1+ KB


In [24]:
#make new DataFrame reflecting object type columns only
objects_df = final_df.select_dtypes(['object'])

#trim values in origianl DataFrame if they exist in the temp "objects" DataFrame
final_df[objects_df.columns] = objects_df.apply(lambda a: a.str.strip())

In [25]:
#identify any posts with empty, [deleted], or [removed] values -> replace with a NULL value
final_df['selftext'].replace("[removed]", np.nan, inplace=True)

In [26]:
final_df['selftext'].replace("[deleted]", np.nan, inplace=True)

In [27]:
final_df['selftext'].replace("", np.nan, inplace=True)

In [28]:
final_df['selftext'].replace(" ", np.nan, inplace=True)

In [29]:
#drop NULLs to remove the bad post data
final_df = final_df.dropna()

2021-01-19 14:27:27.740 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [30]:
#drop any duplicate posts if they exist
final_df.drop_duplicates(subset='selftext', inplace=True)

In [31]:
#check for NULLs - removed in extraction notebook so shouldn't be any
final_df.isnull().sum()

title                 0
author                0
selftext              0
subreddit             0
score                 0
created_utc           0
id                    0
is_fantasyfootball    0
dtype: int64

In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5608 entries, 4 to 96
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               5608 non-null   object
 1   author              5608 non-null   object
 2   selftext            5608 non-null   object
 3   subreddit           5608 non-null   object
 4   score               5608 non-null   int64 
 5   created_utc         5608 non-null   int64 
 6   id                  5608 non-null   object
 7   is_fantasyfootball  5608 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 394.3+ KB


In [33]:
final_df.sort_values(by=['selftext'])

,title,author,selftext,subreddit,score,created_utc,id,is_fantasyfootball
50,A song for my kicker.,Hugh_Bromont,"""Amid the chaos of that day, when all I could ...",fantasyfootball,1,1609184760,klxgi2,1
16,That's what Josh Jacob gets for trolling fanta...,stevendom1987,"""Cuhh KARMA IS A BITCH 😂😂""\n\n[His Tweet](http...",fantasyfootball,1,1609048373,kkyd09,1
81,Sunday Night Clinching Club! Congrats to every...,randomq17,"""On any given Sunday."" A phrase we all know al...",fantasyfootball,1,1608526938,khakct,1
88,ProFootballDoc on CEH injury,HookFL,"""The Chiefs running back was tackled in an awk...",fantasyfootball,1,1608526200,khad7k,1
18,Analyzing the Confusing Minutes of the Nuggets...,LightningNissan,# Clippers\n\n* **IMPORTANT - We know that the...,dfsports,3,1599985557,irujet,0
...,...,...,...,...,...,...,...,...
26,The Kamaracaust,necisizer,~~Five~~ SIX rushing TDs today. It's utter ins...,fantasyfootball,1,1608944091,kk9gdg,1
28,Derrick Henry take our energy energy thread,Duma123,༼ つ ◕_◕ ༽つ DERRICK HENRY TAKE MY ENERGY ༼ つ ◕_...,fantasyfootball,1,1608943967,kk9f6l,1
27,Derrick Henry take our energy thread,Duma123,༼ つ ◕_◕ ༽つ DERRICK HENRY TAKE MY ENERGY ༼ つ ◕_...,fantasyfootball,1,1608944036,kk9fub,1
65,4/9 DK LOL SPREADSHEET,King-Slick,‪The best #DFS #LeagueofLegends spreadsheet. D...,dfsports,1,1586390926,fxi00f,0


In [34]:
#count how many post per subreddit are left after cleanising process
final_df['is_fantasyfootball'].value_counts()

0    3912
1    1696
Name: is_fantasyfootball, dtype: int64

In [35]:
#export merged DataFrame to csv file to use for modeling
final_df.to_csv('../datasets/final_df.csv')